In [0]:
cd /content/drive/My Drive/seg_test/seg_test


/content/drive/My Drive/seg_test/seg_test


In [0]:
!ls


buildings  forest  glacier  mountain  sea  street


In [0]:
!unzip intel-image-classification.zip

In [0]:
import tensorflow as tf
from tensorflow import keras
from keras.utils.vis_utils import model_to_dot
import os
import matplotlib.pyplot as plot
import cv2
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix as CM
from random import randint
from IPython.display import SVG
import matplotlib.gridspec as gridspec
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:

train_datagen = ImageDataGenerator(validation_split=0.2,rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    directory=r'/content/drive/My Drive/seg_train/seg_train/',
    target_size=(150,150),
    class_mode='categorical',
    batch_size=16,
    subset='training'

)
validation_generator = train_datagen.flow_from_directory(
    directory=r'/content/drive/My Drive/seg_train/seg_train/',
    target_size=(150,150),
    class_mode='categorical',
    batch_size=16,
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    directory=r'/content/drive/My Drive/seg_test/seg_test/',
    target_size=(150,150),
    batch_size=1,
    class_mode='categorical'
)



Found 11230 images belonging to 6 classes.
Found 2804 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [0]:
class MyCallBacks(keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs={}):
    if (logs.get('val_acc')>0.8):
      print("\n 80% accuracy reached on validation set so stopping the training")
      self.model.stop_training=True

callbacks = MyCallBacks()

model = keras.Sequential([
    keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(150,150,3)),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(32,(3,3),activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(64,(3,3),activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(128,(3,3),activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(128,(3,3),activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(128,activation='relu'),
    keras.layers.Dense(50,activation='relu'),
    keras.layers.Dropout(rate=0.5),
    keras.layers.Dense(6,activation='softmax')
    
])

model.compile(optimizer=keras.optimizers.Adam(lr=0.0001),loss=keras.losses.CategoricalCrossentropy(from_logits=False),metrics=['accuracy'])


model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 15, 15, 128)      

In [0]:

history = model.fit_generator(
    train_generator,
    steps_per_epoch=175,   
    epochs=30,
    verbose=2,
    validation_data=validation_generator,
    validation_steps=175
)

Epoch 1/30
175/175 - 14s - loss: 0.7790 - accuracy: 0.7261 - val_loss: 0.6527 - val_accuracy: 0.7657
Epoch 2/30
175/175 - 14s - loss: 0.7641 - accuracy: 0.7268 - val_loss: 0.6737 - val_accuracy: 0.7596
Epoch 3/30
175/175 - 14s - loss: 0.7649 - accuracy: 0.7246 - val_loss: 0.6290 - val_accuracy: 0.7800
Epoch 4/30
175/175 - 14s - loss: 0.7585 - accuracy: 0.7300 - val_loss: 0.6285 - val_accuracy: 0.7718
Epoch 5/30
175/175 - 14s - loss: 0.7317 - accuracy: 0.7452 - val_loss: 0.6297 - val_accuracy: 0.7875
Epoch 6/30
175/175 - 14s - loss: 0.7087 - accuracy: 0.7557 - val_loss: 0.6143 - val_accuracy: 0.7829
Epoch 7/30
175/175 - 14s - loss: 0.7505 - accuracy: 0.7307 - val_loss: 0.6294 - val_accuracy: 0.7754
Epoch 8/30
175/175 - 14s - loss: 0.6782 - accuracy: 0.7639 - val_loss: 0.6033 - val_accuracy: 0.7900
Epoch 9/30
175/175 - 14s - loss: 0.6642 - accuracy: 0.7695 - val_loss: 0.6399 - val_accuracy: 0.7686
Epoch 10/30
175/175 - 14s - loss: 0.7044 - accuracy: 0.7596 - val_loss: 0.6167 - val_accura

In [0]:
model.evaluate_generator(test_generator,verbose=1,steps=3000)

3000/3000 [==============================] - 16s 5ms/step - loss: 0.5172 - accuracy: 0.8173


[0.5172491669654846, 0.8173333406448364]